In [7]:
# https://github.com/sammchardy/python-binance/blob/master/docs/account.rst
# Run this cell first. Only works on specific ip addresses.

import threading
from decimal import *
from binance.client import Client
client = Client("") # removed private keys


def checkAccBalances():
    prices = client.get_all_tickers()
    info = client.get_account()
    balances = info['balances']
    for coin in balances:
        total = Decimal(coin['free']) + Decimal(coin['locked'])
        if total > 0.0001:
            print(str(coin['asset']) + ": " + str(total))
            symbol = coin["asset"] + "BTC"

            btcValue = -1
            usdValue = -1
            
            if symbol == "BTCBTC":
                btcValue = total
            elif symbol == "USDTBTC":
                usdValue = total
                btcValue = 0
            
            for listing in prices:
                if listing['symbol'] == symbol:
                    btcValue = float(total) * float(listing['price'])
            if btcValue == -1:
                print("Find Symbol Price Error")
            
            for listing in prices:
                if listing['symbol'] == "BTCUSDT" and btcValue:
                    usdValue = btcValue * float(listing['price'])

            print("$" + str(usdValue))
checkAccBalances()


BNB: 4.67907147
$74.13809059916132
USDT: 148.42784744
$148.42784744
ZRX: 63.93600000
$47.85867639302399
WTC: 0.94233000
$6.473336318528309
IOTA: 21.97800000
$22.7323805291352
QSP: 398.60200000
$28.001223847396798
ICX: 17.26272000
$31.207468983828484


In [3]:
# This cell was for a market sell order (WTC to BNB) if btc drops below 6598
global btcPrice
btcPrice = 0
def checkBtc():
    global btcPrice
    prices = client.get_all_tickers()

    threading.Timer(15.0, checkBtc).start()
    
    for listing in prices:
        if listing['symbol'] == "BTCUSDT":
            newBtcPrice = float(listing['price'])
    if btcPrice and newBtcPrice == btcPrice:
        return
    btcPrice = newBtcPrice
    print("BTC: $" + str(btcPrice))
    if btcPrice < 6598:
        order = client.order_market_sell(
        symbol='WTCBNB',
        quantity=1)
        print("\nSOLD 1 WTC\n\n")
        checkAccBalances()
    else:
        print("Waiting for BTC < 6599 to Sell\n")

checkBtc()



BTC: $6615.33
Waiting for BTC < 6599 to Sell

BTC: $6617.68
Waiting for BTC < 6599 to Sell

BTC: $6613.4
Waiting for BTC < 6599 to Sell

BTC: $6614.75
Waiting for BTC < 6599 to Sell

BTC: $6617.67
Waiting for BTC < 6599 to Sell

BTC: $6613.51
Waiting for BTC < 6599 to Sell

BTC: $6617.68
Waiting for BTC < 6599 to Sell

BTC: $6614.67
Waiting for BTC < 6599 to Sell

BTC: $6617.68
Waiting for BTC < 6599 to Sell

BTC: $6614.7
Waiting for BTC < 6599 to Sell

BTC: $6617.95
Waiting for BTC < 6599 to Sell

BTC: $6617.68
Waiting for BTC < 6599 to Sell

BTC: $6614.67
Waiting for BTC < 6599 to Sell

BTC: $6617.0
Waiting for BTC < 6599 to Sell

BTC: $6614.67
Waiting for BTC < 6599 to Sell

BTC: $6616.94
Waiting for BTC < 6599 to Sell

BTC: $6614.0
Waiting for BTC < 6599 to Sell

BTC: $6616.92
Waiting for BTC < 6599 to Sell

BTC: $6615.0
Waiting for BTC < 6599 to Sell

BTC: $6616.92
Waiting for BTC < 6599 to Sell

BTC: $6615.0
Waiting for BTC < 6599 to Sell

BTC: $6613.35
Waiting for BTC < 6599 to 

Exception in thread Thread-189:
Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/anaconda3/lib/python3.6/threading.py", line 1182, in run
    self.function(*self.args, **self.kwargs)
  File "<ipython-input-3-0be4207540b9>", line 21, in checkBtc
    checkAccBalances()
NameError: name 'checkAccBalances' is not defined



BTC: $6602.0
Waiting for BTC < 6599 to Sell

BTC: $6600.0
Waiting for BTC < 6599 to Sell

BTC: $6601.94
Waiting for BTC < 6599 to Sell

BTC: $6600.0
Waiting for BTC < 6599 to Sell

BTC: $6599.98
Waiting for BTC < 6599 to Sell

BTC: $6599.06
Waiting for BTC < 6599 to Sell

BTC: $6598.47
Waiting for BTC < 6599 to Sell

BTC: $6600.0
Waiting for BTC < 6599 to Sell

BTC: $6596.45

SOLD 1 WTC




Exception in thread Thread-201:
Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/anaconda3/lib/python3.6/threading.py", line 1182, in run
    self.function(*self.args, **self.kwargs)
  File "<ipython-input-3-0be4207540b9>", line 21, in checkBtc
    checkAccBalances()
NameError: name 'checkAccBalances' is not defined



In [9]:
# This cell is to cover my short position and buy WTC from BNB when btc rises 1% after going below 6250.
# I didn't leave this running overnight, but btc dropped to 5950 and is at 6135 right now. 
global btcPrice, recentLow
btcPrice = 0
recentLow = 10000000
def checkBtc():
    global btcPrice
    global recentLow
    prices = client.get_all_tickers()
    
    if recentLow < 6250:
        refreshSeconds = 1
    else:
        refreshSeconds = 15
        
    threading.Timer(refreshSeconds, checkBtc).start()
    
    for listing in prices:
        if listing['symbol'] == "BTCUSDT":
            newBtcPrice = float(listing['price'])
    if btcPrice and newBtcPrice == btcPrice:
        return
    btcPrice = newBtcPrice
    if btcPrice < recentLow:
        recentLow = btcPrice
        print("recentLow: " + str(recentLow))
        print("Target: $" + str(recentLow * 1.01))
        print("Trigger: Btc < 6250")
    print("\nBTC: $" + str(btcPrice))
    
    if btcPrice < 6250 and btcPrice > recentLow * 1.01:
        print("\nBought 1 WTC\n\n")
        order = client.order_market_buy(
        symbol='WTCBNB',
        quantity=1)
        #checkAccBalances()
    #else:
        #print("Waiting for BTC < 6599 to Sell\n")

checkBtc()


BTC: $6586.34
recentLow: 6586.33
Target: $6652.1933
Trigger: Btc < 6250

BTC: $6586.33
recentLow: 6582.0
Target: $6647.82
Trigger: Btc < 6250

BTC: $6582.0

BTC: $6586.31

BTC: $6586.32

BTC: $6582.0

BTC: $6586.31
recentLow: 6581.21
Target: $6647.0221
Trigger: Btc < 6250

BTC: $6581.21

BTC: $6586.0
recentLow: 6579.01
Target: $6644.8001
Trigger: Btc < 6250

BTC: $6579.01

BTC: $6585.99

BTC: $6580.26

BTC: $6585.99

BTC: $6585.98

BTC: $6585.96

BTC: $6585.97

BTC: $6580.28

BTC: $6585.97

BTC: $6586.0

BTC: $6580.74

BTC: $6585.99

BTC: $6585.96

BTC: $6580.76

BTC: $6580.77

BTC: $6585.99

BTC: $6580.82

BTC: $6581.01

BTC: $6586.0

BTC: $6581.03

BTC: $6585.99

BTC: $6586.0

BTC: $6581.03

BTC: $6581.01

BTC: $6584.5

BTC: $6581.02

BTC: $6585.99

BTC: $6580.88

BTC: $6580.81

BTC: $6580.88

BTC: $6585.99

BTC: $6580.81

BTC: $6580.26

BTC: $6585.97

BTC: $6580.22

BTC: $6580.26

BTC: $6580.81

BTC: $6580.88

BTC: $6580.22

BTC: $6585.96

BTC: $6585.95

BTC: $6580.22

BTC: $6583.4